%%markdown

# GigaSpaces Task Examples
This notebook will demonstrate the basics of the GigaSpaces task API.



%%markdown

##### GigaSpaces start up instructions

1. In a console window, go to the GigaSpaces install directory/bin (for convenience GS_HOME has already been defined).

`cd $GS_HOME/bin`

2. Start the service grid with a manager and 4 GSCs (in developer mode)

`./gs.sh host run-agent --auto --gsc=4`

3. In another console window, deploy a space named demo.

`./gs.sh space deploy --partitions=2 --ha demo`


In [1]:
%%pom
repositories:
  - id: org.openspaces
    url: http://maven-repository.openspaces.org
dependencies:
  - groupId: org.gigaspaces
    artifactId: xap-openspaces
    version: 16.4.0

  - groupId: com.gigaspaces.training
    artifactId: model
    version: 1.0
        

In [2]:
%%classpath

${JUPYTER_DATA_DIR}/kernels/ganymede-2.1.2-java-11-gs-dev-lab-3.3.3/kernel.jar
${user.home}/.m2/repository/org/gigaspaces/xap-openspaces/16.4.0/xap-openspaces-16.4.0.jar
${user.home}/.m2/repository/org/gigaspaces/xap-datagrid/16.4.0/xap-datagrid-16.4.0.jar
${user.home}/.m2/repository/io/opentracing/opentracing-util/0.33.0/opentracing-util-0.33.0.jar
${user.home}/.m2/repository/io/opentracing/opentracing-api/0.33.0/opentracing-api-0.33.0.jar
${user.home}/.m2/repository/io/opentracing/opentracing-noop/0.33.0/opentracing-noop-0.33.0.jar
${user.home}/.m2/repository/com/github/oshi/oshi-core/5.5.0/oshi-core-5.5.0.jar
${user.home}/.m2/repository/net/java/dev/jna/jna-platform/5.6.0/jna-platform-5.6.0.jar
${user.home}/.m2/repository/org/gigaspaces/xap-common/16.4.0/xap-common-16.4.0.jar
${user.home}/.m2/repository/org/gigaspaces/xap-asm/16.4.0/xap-asm-16.4.0.jar
${user.home}/.m2/repository/org/gigaspaces/xap-trove/16.4.0/xap-trove-16.4.0.jar
${user.home}/.m2/repository/org/xerial/sqlite-jdbc/3

In [3]:
%%env

#Wed Feb 28 10:50:42 EST 2024
PATH=/home/dixson/.local/bin\:/usr/local/sbin\:/usr/local/bin\:/usr/sbin\:/usr/bin\:/sbin\:/bin\:/usr/games\:/usr/local/games\:/snap/bin\:/home/dixson/work/tools/apache-maven-3.8.3/bin\:/home/dixson/work/tools/virt/google-cloud-sdk/bin\:/home/dixson/.local/bin\:
XMODIFIERS=@im\=ibus
GDMSESSION=ubuntu
DBUS_SESSION_BUS_ADDRESS=unix\:path\=/run/user/1000/bus
DEFAULTS_PATH=/usr/share/gconf/ubuntu.default.path
SSH_AGENT_PID=1528
COLORTERM=truecolor
USERNAME=dixson
SESSION_MANAGER=local/dixson-pc\:@/tmp/.ICE-unix/1603,unix/dixson-pc\:/tmp/.ICE-unix/1603
MANAGERPID=1109
PWD=/home/dixson/work/project/21
GJS_DEBUG_TOPICS=JS ERROR;JS LOG
LESSOPEN=| /usr/bin/lesspipe %s
GNOME_DESKTOP_SESSION_ID=this-is-deprecated
GNOME_TERMINAL_SCREEN=/org/gnome/Terminal/screen/2b768596_f490_49f0_bb6f_a9984415a064
GTK_MODULES=gail\:atk-bridge
SHLVL=1
TERM=xterm-256color
XDG_CONFIG_DIRS=/etc/xdg/xdg-ubuntu\:/etc/xdg
JPY_PARENT_PID=32815
JUPYTER_CONFIG_DIR=/home/dixson/.jupyter
_=/home

In [4]:
%%java

import com.gigaspaces.async.AsyncFuture;
import com.gigaspaces.async.AsyncFutureListener;
import com.gigaspaces.async.AsyncResult;


import org.openspaces.core.GigaSpace;
import org.openspaces.core.GigaSpaceConfigurer;
import org.openspaces.core.space.SpaceProxyConfigurer;

import org.openspaces.core.executor.DistributedTask;

import java.util.List;



In [5]:
%%java

// define the DistributedTask
public class MyDistTask implements DistributedTask<Integer, Long>, java.io.Serializable {
    private static final long serialVersionUID = 0L;
    
    public Integer execute() throws Exception {
        return 1;
    }

    @Override
    public Long reduce(List<AsyncResult<Integer>> results) throws Exception {
        long count = 0;
        for (AsyncResult<Integer> result : results) {
            if (result.getException() != null) {
                throw result.getException();
            }
            count += result.getResult();
        }
        return count;
    }
}


In [6]:
%%java

public class MyFutureListener implements AsyncFutureListener<Long> {
    @Override
    public void onResult (AsyncResult <Long> asyncResult) {
        if (asyncResult.getException() != null) {
            try {
                throw asyncResult.getException();
            } catch (Exception e) {
                e.printStackTrace();
            }
        }
    }
}

In [7]:
%%java
// an example of instantiating a remote proxy
SpaceProxyConfigurer configurer = new SpaceProxyConfigurer("demo");
GigaSpace gigaSpace = new GigaSpaceConfigurer(configurer).gigaSpace();

AsyncFuture<Long> future = gigaSpace.execute(new MyDistTask());

future.setListener(new MyFutureListener());

long result = future.get();

System.out.println("Result is: " + result);


2024-02-28 10:50:44,493 CONFIG [com.gigaspaces.logger] - Log file: /home/dixson/work/project/21/gs-dev-jupyter-lab/logs/2024-02-28~10.50-gigaspaces-service-127.0.1.1-88638.log
Result is: 2
